In [ ]:
import csv
import requests
import time
import base64
import pandas as pd
import numpy as np

In [ ]:
# 读取CSV文件
data = pd.read_csv('repos_data_2000.csv')

print(data.tail(10))
data_len = len(data)
print("初始数据量为：", data_len)
# 去掉无标签数据行
no_topic_list = []
for i in range(data_len):
    if data['topics'][i] == '[]':
        no_topic_list.append(i)
        #print(data['ID'][i],data['name'][i],data['topics'][i])


In [ ]:
print('无标签数据量为：', len(no_topic_list))
data_new=data.drop(no_topic_list) # 删除 no_topic_list 行数据
data_new.to_csv("./data_new1.csv",index=0)

In [ ]:
# 读取CSV文件
data_new1 = pd.read_csv('data_new1.csv')

print(data_new1.tail(10))
data_len1 = len(data_new1)
print("有标签数据量为：", data_len1)
# 去掉单一标签数据行
one_topic_list = []
for i in range(data_len1):
    if len(eval(data_new1['topics'][i])) == 1:
        one_topic_list.append(i)
        #print(data_new1['ID'][i],data_new1['name'][i],data_new1['topics'][i])

In [ ]:
print('单标签数据量为：', len(one_topic_list))
data_new=data_new1.drop(one_topic_list) # 删除 one_topic_list 行数据
data_new.to_csv("./data_new2.csv",index=0)

In [ ]:
data_new2 = pd.read_csv('data_new2.csv')
print(data_new2.tail(10))
data_len2 = len(data_new2)
print("多标签数据量为：", data_len2)
# 去掉 ID 重复数据行（每次换 search 条件时数据可能会有所增减）
ID_list = []
duplicate_list = []
for i in range(data_len2):
    if data_new2['ID'][i] in ID_list:
        duplicate_list.append(i)
        #print(data_new2['ID'][i],data_new2['name'][i],data_new2['topics'][i])
    ID_list.append(data_new2['ID'][i])

In [ ]:
print('ID 重复数据量为：', len(duplicate_list))
data_new=data_new2.drop(duplicate_list) # 删除 duplicate_list 行数据
data_new.to_csv("./data_new3.csv",index=0)

In [ ]:
data_new3 = pd.read_csv('data_new3.csv')
data_len3 = len(data_new3)
print("清洗后的数据量为：", data_len3)
# 提取每个仓库的标签，并另存为一个文件
topics_list = data_new3['topics']
print(topics_list)
csv_filename = 'origin_topics.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(data_len3):
        f.write("\"")
        f.write(str(topics_list[i]))
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
topic_set = set()
count = 0
for i in range(data_len3):
    topic_list = eval(topics_list[i])
    for item in topic_list:
        topic_set.add(item)
        count +=1 # 每个仓库的初始标签没有重复项
print('初始不同标签数量：', len(topic_set))
print('标签总数：', count)
print('初始各标签平均出现频率：', round(count/len(topic_set),2))

In [ ]:
# 标签统一化（实体消歧）
import json

file = open("rules/annotated_github_topics_wikidata.json", 'r', encoding='utf-8')
wiki = []
for line in file.readlines():
    dic = json.loads(line)
    wiki.append(dic)
wiki_dict = eval(str(wiki))
#print(wiki[1])
print('GitHub 标题的 wiki 百科大小为：',len(wiki_dict))
#print(wiki_dict[1]['Wikidata Title']) # 字符串类型（只有一个）
#print(wiki_dict[1]['GitHub Topic']) # list 类型
#print(wiki_dict[1]['Wikidata Aliases']) # list 类型


In [ ]:
# 数据截选
total_topic = []

for i in range(data_len3):
    topic_list = eval(topics_list[i])
    repo_topic = []
    for item in topic_list: # 对第 i+1 个仓库的每个标签
        for j in range(len(wiki_dict)):
            wiki_j = wiki_dict[j]['GitHub Topic'] + [wiki_dict[j]['Wikidata Title']] + wiki_dict[1]['Wikidata Aliases']
            if item in wiki_j:
                #print('第',i+1,'个仓库的主题标签',item,'为别称，替换为',wiki_j[0])
                item = wiki_j[0] # 将 item 视为别称，并直接规范为 GitHub Topic 的第一个
                break
        repo_topic.append(item) 
    repo_topic = list(set(repo_topic))
    total_topic.append(repo_topic)   
#print(total_topic[3])      

In [ ]:
topic_set1 = set()
count1 = 0
for i in range(data_len3):
    topic_list = set(total_topic[i])
    topic_set1 = topic_set1.union(topic_list)
    count1 += len(topic_list)
print('wiki 处理后不同标签数量：', len(topic_set1))
print('wiki 处理后标签总数：', count1)
print('wiki 处理后各标签平均出现频率：', round(count1/len(topic_set1),2))

In [ ]:
# 存储经 wiki 百科资料处理过的主题标签数据
csv_filename = 'topics1.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(data_len3):
        f.write("\"")
        f.write(str(total_topic[i]))
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
topic_csv = pd.read_csv('topics1.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']

In [ ]:
# 去除版本号
rule1 = pd.read_csv('rules/topics_contains_version.csv')
rule_len1 = len(rule1)
print("该规则数据量为：", rule_len1)
total_topic1 = []
for i in range(topic_csv_len):
    new_topic_list = []
    topic_list = eval(topics_list[i])
    for item in topic_list:
        flag = 0
        for j in range(rule_len1):
            if rule1['version'][j]==item: # 匹配上版本号
                print(i,"个主题列中，",item,"是一个版本号，需修改")
                if rule1['topic'][j]=='-1': # 是版本号但无法修改，保留item
                    print("该版本号不做修改")
                    break
                else:
                    item = (rule1['topic'][j])
                    print("修改为", item)
                    if item[0] == '[':
                        flag = 1
                        item = eval(item)
                    break
        if flag == 1:
            new_topic_list.extend(item)
            continue
        new_topic_list.extend([item])
    total_topic1.append(new_topic_list)
#print(total_topic1)

In [ ]:
# 存储经版本号处理过的主题标签数据
csv_filename = 'topics2.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(total_topic1[i]))
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
topic_csv = pd.read_csv('topics2.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']

In [ ]:
# 去除标签中的数字
rule2 = pd.read_csv('rules/topics_contains_number.csv')
rule_len2 = len(rule2)
print("该规则数据量为：", rule_len2)
total_topic2 = []
for i in range(topic_csv_len):
    new_topic_list = []
    topic_list = eval(topics_list[i])
    for item in topic_list:
        #flag = 0
        for j in range(rule_len2):
            if rule2['number'][j]==item: # 匹配上
                print(i,"个主题列中，",item,"是一个带数字标签，需修改为",rule2['topic'][j])
                item = rule2['topic'][j]
                break
        new_topic_list.append(item)
    total_topic2.append(new_topic_list)
print(total_topic2)

In [ ]:
# 存储经数字去除处理过的主题标签数据
csv_filename = 'topics3.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(total_topic2[i]))
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
topic_csv = pd.read_csv('topics3.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
print(topics_list)

In [ ]:
# 去掉版本和数字处理后重复出现的标签
topic_set3 = set()
count3 = 0
topic3 = []
for i in range(topic_csv_len):
    topic_list = set(eval(topics_list[i]))
    topic3.append(list(topic_list))
    topic_set3 = topic_set3.union(topic_list)
    count3 += len(topic_list)
print('version 和 number 处理后不同标签数量：', len(topic_set3))
print('version 和 number 处理后标签总数：', count3)
print('version 和 number 处理后各标签平均出现频率：', round(count3/len(topic_set3),2))

In [ ]:
# 存储去重处理过的主题标签数据
csv_filename = 'topics4.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(topic3[i]))
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
## 修改 rules
# 词汇清洗条件筛选
rule1 = pd.read_csv('rules/delete.csv')
#rule2 = pd.read_csv('rules/abstract.csv')
rule_len1 = len(rule1)
#rule_len2 = len(rule2)
print("该规则数据量为：", rule_len1)#,rule_len2)
number = []
#for i in range(rule_len2):
    #for j in range(i+1,rule_len2):
    #    if (rule2['topic'][i]==rule2['topic'][j]):# and (rule1['topics'][i] in list(rule2['topic'])):
    #        number.append(i)
    #        break
#print(number)
#rule_1 = rule2.iloc[number[10:]]
#print(rule_1)
#rule1=rule1.drop(number[:7]) # 删除 duplicate_list 行数据
rule1[['del']].to_csv("rules/delete.csv",index=0)

In [ ]:
# 长标签分解关键词
rule3 = pd.read_csv('rules/split_dash_topics.csv')
rule_len3 = len(rule3)
print("该规则数据量为：", rule_len3)
topic_csv = pd.read_csv('topics4.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
t_oreder = ['t1','t2','t3','t4','t5'] ###
new_total_topic = []
for i in range(topic_csv_len):
    one_topic_list = eval(topics_list[i])
    new_topic_list = []
    for item in one_topic_list:
        flag = 0
        item_len = len(item.split('-'))
        if (item_len)>1:
            for j in range(rule_len3):
                if item == rule3['total'][j]: # 找到，拆解，退出该词查找
                    print(i,'仓库中的',item,"需拆解")
                    re_item = []
                    for k in range(5): ###
                        if str(rule3[t_oreder[k]][j])!='nan':
                            re_item.append(rule3[t_oreder[k]][j])
                    #print(re_item)
                    flag = 1
                    new_topic_list.extend(re_item)
                    break
        if flag == 0:
            new_topic_list.append(item)
    no_same = set(new_topic_list)
    new_total_topic.append(list(no_same))
print(new_total_topic)

In [ ]:
# 标签失误处理：找出被错误处理的仓库id
topic_csv = pd.read_csv('topics4.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
wrong_topic_id = []
for i in range(topic_csv_len):
    one_topic_list = eval(topics_list[i])
    new_topic_list = []
    for item in one_topic_list:       
        divid = item.split('-')
        item_len = len(divid)
        if (item_len)>2:
            if divid[2] == 'code' and divid[1] == 'studio' and divid[0]=='visual': # 错误拆解词
                print(i,'仓库中的',item,"需补充")
                wrong_topic_id.append(i)
                break
print(wrong_topic_id)
print(len(wrong_topic_id))

In [ ]:
# 标签失误处理：根据仓库id修正错误
topic_csv = pd.read_csv('topics8.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
correct_topic = []
count = 0
for i in range(topic_csv_len):
    one_topic_list = eval(topics_list[i])
    new_topic_list = []
    flag = 0
    if i in wrong_topic_id:       
        print(i,'仓库中的错误标签需添加')
        one_topic_list.append('vscode') # 添加被错误删除的词
        print(one_topic_list)
        count += 1
    correct_topic.append(one_topic_list)
print(len(correct_topic))
print(count)

In [ ]:
# 存储错误处理修改回来后的标签
csv_filename = 'topics8.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(correct_topic[i])) ###new_total_topic
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 对抽象和复数标签进行转换
rule4 = pd.read_csv('rules/abstract_plural_topics.csv')
rule_len4 = len(rule4)
print("该规则数据量为：", rule_len4)
topic_csv = pd.read_csv('topics5.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
new_total_topic = []
for i in range(topic_csv_len):
    one_topic_list = eval(topics_list[i])
    new_topic_list = []
    for item in one_topic_list:
        flag = 0
        for j in range(rule_len4):
            if item == rule4['abstract'][j]: # 找到，转换，退出该词查找
                print(i,'仓库中的',item,"需转换")
                new_topic_list.append(rule4['topic'][j])
                flag = 1
                break
        if flag == 0:
            new_topic_list.append(item)
    new_total_topic.append(list(set(new_topic_list)))
print(new_total_topic)

In [ ]:
# 存储抽象和复数标签转换过的主题标签数据
csv_filename = 'topics6.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(new_total_topic[i])) 
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 去除停用词
rule5 = pd.read_csv('rules/remove_stopwords_topic.csv')
rule_len5 = len(rule5)
print("该规则数据量为：", rule_len5)
topic_csv = pd.read_csv('topics6.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
new_total_topic = []
for i in range(topic_csv_len):
    one_topic_list = eval(topics_list[i])
    new_topic_list = []
    for item in one_topic_list:
        if item not in list(rule5['stop_word']):
            new_topic_list.append(item)
        else:
            print(i,'仓库中的',item,"被移除")
    new_total_topic.append(list(set(new_topic_list)))
print(new_total_topic)

In [ ]:
# 存储去除停用词后的主题标签数据
csv_filename = 'topics7.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(new_total_topic[i])) 
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 去除非标签词汇
rule6 = pd.read_csv('rules/delete.csv')
rule_len6 = len(rule6)
print("该规则数据量为：", rule_len6)
topic_csv = pd.read_csv('topics7.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
new_total_topic = []
for i in range(topic_csv_len):
    one_topic_list = eval(topics_list[i])
    new_topic_list = []
    for item in one_topic_list:
        if item not in list(rule6['del']):
            new_topic_list.append(item)
        else:
            print(i,'仓库中的',item,"被移除")
    new_total_topic.append(list(set(new_topic_list)))
print(new_total_topic)

In [ ]:
# 存储去除非标签词汇后的主题标签数据
csv_filename = 'topics8.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(new_total_topic[i])) 
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 再次对标签进行wiki清洗
import json
file = open("rules/annotated_github_topics_wikidata.json", 'r', encoding='utf-8')
wiki = []
for line in file.readlines():
    dic = json.loads(line)
    wiki.append(dic)
wiki_dict = eval(str(wiki))
print('GitHub 标题的 wiki 百科大小为：',len(wiki_dict))
topic_csv = pd.read_csv('topics8.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
total_topic = []
change = set()
for i in range(topic_csv_len):
    topic_list = eval(topics_list[i])
    repo_topic = []
    for item in topic_list: # 对第 i+1 个仓库的每个标签
        for j in range(len(wiki_dict)):
            wiki_j = wiki_dict[j]['GitHub Topic'] + [wiki_dict[j]['Wikidata Title']] + wiki_dict[1]['Wikidata Aliases']
            if (item in wiki_j) and (item != wiki_j[0]):
                print('第',i+1,'个仓库的主题标签',item,'为别称，替换为',wiki_j[0])
                item = wiki_j[0] # 将 item 视为别称，并直接规范为 GitHub Topic 的第一个
                change.add(item)
                break
        repo_topic.append(item) 
    repo_topic = list(set(repo_topic))
    total_topic.append(repo_topic) 
print(change)
print(total_topic)

In [ ]:
# 存储wiki清洗后的主题标签数据
csv_filename = 'topics9.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(total_topic[i])) 
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 各仓库标签去重
topic_csv = pd.read_csv('topics4.csv')
topic_csv_len = len(topic_csv)
print("主题数据量为：", topic_csv_len)
topics_list = topic_csv['topics']
new_total_topic = []
for i in range(topic_csv_len):
    one_topic_list = eval(topics_list[i])
    new_total_topic.append(list(set(one_topic_list)))
print(new_total_topic[1])
csv_filename = 'topics4.csv'
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('topics\n')
    for i in range(topic_csv_len):
        f.write("\"")
        f.write(str(new_total_topic[i])) 
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")